In [1]:
using Laplacians

### first gaus siedel and jacobi - these seem to be pretty unefficient

In [2]:
include("../../../src/preconditioners.jl")

gs (generic function with 1 method)

In [3]:
n = 10000
a = chimera(n,1);
la = lap(a)
b = rand(n); b = b - mean(b);

In [4]:
@time f = gs(la);

  0.023752 seconds (18.99 k allocations: 999.666 KB)


In [5]:
@time x = pcg(la, b, f, tol = 1e-6, maxits = 10000, verbose=true);

PCG stopped after: 1536 iterations with relative error 9.935877112196036e-7.
  2.566890 seconds (31.28 M allocations: 1.282 GB, 5.89% gc time)


In [6]:
norm(la * x - b) / norm(b)

9.9358769943904e-7

In [7]:
@time f = jacobi(la);

  0.002542 seconds (525 allocations: 109.604 KB)


In [8]:
@time x = pcg(la, b, f, tol = 1e-6, maxits = 10000, verbose=true);

PCG stopped after: 3347 iterations with relative error 9.925893705395602e-7.
  1.824041 seconds (150.90 k allocations: 1.251 GB, 8.66% gc time)


In [9]:
norm(la * x - b) / norm(b)

9.925893328174032e-7

### let's take a look at the algebraic multigrid solver

In [10]:
using PyAMG

In [11]:
@time x = PyAMG.solve(la, b, tol = 1e-7, maxiter = 100);

  1.892915 seconds (1.12 M allocations: 47.563 MB, 0.99% gc time)
  Detected a Hermitian matrix
    maxiter = 100
    iteration 1
    iteration 2
    iteration 3
    iteration 4
    iteration 5
    iteration 6
    iteration 7
    iteration 8
    iteration 9
    iteration 10
    iteration 11
    iteration 12
    iteration 13
    iteration 14
    iteration 15
    iteration 16
  Residual reduction ||r_k||/||r_0|| = 6.20e-08


/Users/serbanstan/.julia/v0.4/Conda/deps/usr/lib/python2.7/site-packages/pyamg/strength.py:533: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if B == 'ones':


In [12]:
norm(la * x - b) / norm(b)

1.8948933978366254e-7

In [13]:
?PyAMG.solve

`solve(amg::AMGSolver, b, kwargs...)`

Returns a `Vector` with the result of the AMG solver.

### `kwargs`  (copy-pasted from Python docs)

  * `x0` : Initial guess.
  * `tol` : Stopping criteria: relative residual r[k]/r[0] tolerance.
  * `maxiter` : Stopping criteria: maximum number of allowable iterations.
  * `cycle` : {"V","W","F","AMLI"}     Type of multigrid cycle to perform in each iteration.
  * `accel` : Defines acceleration method.  Can be a string such as "cg"     or "gmres" which is the name of an iterative solver in     `pyamg.krylov` (preferred) or scipy.sparse.linalg.isolve.     If accel is not a string, it will be treated like a function     with the same interface provided by the iterative solvers in SciPy.         (the function version is not tested in Julia!)
  * `callback` : User-defined function called after each iteration.  It is     called as callback(xk) where xk is the k-th iterate vector.
  * `residuals` : List to contain residual norms at each iteration.

`solve(A::SparseMatrixCSC, b::Vector; kwargs...)`:

PyAMG's 'blackbox' solver. See `pyamg.solve?` for `kwargs`.


### now, let's test on harder graphs

In [13]:
a = readFromFile("3.txt");

In [14]:
n = a.n

90000

In [15]:
la = lap(a);
b = rand(n); b = b - mean(b);

In [18]:
@time x = PyAMG.solve(la, b, tol = 1e-6, maxiter = 200);

 10.239151 seconds (221 allocations: 8.230 MB)
  Detected a Hermitian matrix
    maxiter = 200
    iteration 1
    iteration 2
    iteration 3
    iteration 4
    iteration 5
    iteration 6
    iteration 7
    iteration 8
    iteration 9
    iteration 10
    iteration 11
    iteration 12
    iteration 13
    iteration 14
    iteration 15
    iteration 16
    iteration 17
    iteration 18
    iteration 19
    iteration 20
    iteration 21
    iteration 22
    iteration 23
    iteration 24
    iteration 25
    iteration 26
    iteration 27
    iteration 28
    iteration 29
    iteration 30
    iteration 31
    iteration 32
    iteration 33
    iteration 34
    iteration 35
    iteration 36
    iteration 37
    iteration 38
    iteration 39
    iteration 40
    iteration 41
    iteration 42
    iteration 43
    iteration 44
    iteration 45
    iteration 46
    iteration 47
    iteration 48
    iteration 49
    iteration 50
    iteration 51
    iteration 52
    iteration 53
    iteration

In [19]:
norm(la * x - b) / norm(b)

500682.60852057574

### now, our solver

In [20]:
include("../../../src/samplingTreeSolver.jl");

In [27]:
@time f = samplingSolver(a, tol = 1e-6, maxits = 1000, eps = 0.5, sampConst = 0.02, beta = 1000.0);

rho = 10.41060304582247
Average number of multiedges = 5.2266946715106295
Maximum number of multiedges = 10.410603085816941
Time to build the tree and compute the stretch: elapsed time: 0.827161859 seconds

The total size of the linked list data structure should be at most 4612282
The actual size is 107820

  1.935852 seconds (14.61 M allocations: 866.909 MB, 16.78% gc time)


In [28]:
@time norm(la * f(b) - b) / norm(b)

  1.458242 seconds (25.56 M allocations: 979.914 MB, 10.65% gc time)


8.774380177101899e-7

In [30]:
r = lap(a) * f(b) - b
t = r' * f(r)
t / (b'*f(b))

1x1 Array{Float64,2}:
 3.12443e-15

In [23]:
la = lap(a);

In [24]:
f = gs(la);

In [25]:
@time x = pcg(la, b, f, tol = 1e-2, maxits = 10000, maxtime=20, verbose=true);

PCG stopped at maxtime.
PCG stopped after: 1379 iterations with relative error 1.2248758503253374.
 20.051198 seconds (248.48 M allocations: 10.182 GB, 8.15% gc time)


In [26]:
norm(la * x - b) / norm(b)

1.224875850325474